# Weekly assignment 6: Constrained Newton method

In this exercise you will code up and investigate three solution methods which are
extremely useful in computational methods, including the
midterm assignment.  The methods are:

- bisection method for solving non-linear equations within a closed interval  
- Newton-Raphson method for solving non-linear equations on real line  
- combining the two, the robust Newton algorithm for quick solving non-linear
  equations one open or close intervals  


Recall that Newton method is preferred due to its quadratic convergence, but is only
locally convergent, meaning that if started far enough from the solution, may not converge.

Bisection method is very stable, but requires an interval with alternating signs of
the function value at its ends, and is converging much slower.

We can combine the advantages of the two methods in a polyalgorithm, which is particularly
helpful in complex cases, such as the equations in logs below.

In addition you will learn about a very useful technique called *callback function*, that
allows to use the same code for fast production runs and debugging runs depending on whether
a diagnostic function is passed to the method as an argument.

## Task 1. Bisections

Implement the following algorithm:

- Main inputs: function object/handle, lower and upper bound  
- Additional inputs: maximum number of iterations, convergence tolerance, callback function (None by default)  


1. Check that the signs of the function evaluated at the lower and upper bounds are opposite,
  otherwise raise ValueError with appropriate message.  
1. Initialize iteration counter at 0, set current interval bounds to the initial lower and upper bounds,
  and loop through steps 3-7 for at most maxiter iterations:  
1. Find midpoint of the current interval  
1. If callback function handle is passed, call this function with parameters as described in the starter code  
1. Compute the sign of the function evaluated at the midpoint  
1. Move the bound of the current interval with the same sign to the midpoint and update the current interval  
1. Check for convergence: if the length of the current interval is below the tolerance, exit the loop  
1. Use for-else construct to check if the maximum number of iterations has been achieved, in which case raise
  the RuntimeError with an appropriate message.  
1. Return the midpoint of the current interval as a solution  


Hint: see [https://docs.python.org/3/tutorial/errors.html](https://docs.python.org/3/tutorial/errors.html) for how to raise errors.

In [ ]:
import numpy as np
def bisection(fun, lower, upper, tol=1e-6, maxiter=100, callback=None):
    '''Bisection method to solve fun(x)=0,
       assuming root is between lower and upper.
       Callback function arguments (iter,x,lower,upper) where bounds refer to current interval
    '''
    # write your code here

In [ ]:
import numpy as np
def bisection(fun, lower, upper, tol=1e-6, maxiter=100, callback=None):
    '''Bisection method to solve fun(x)=0,
       assuming root is between lower and upper.
       Callback function arguments (iter,x,lower,upper) where bounds refer to current interval
    '''
    if fun(lower)*fun(upper)>0:
        raise(ValueError('Bad initial lower and upper limits'))
    for iter in range(maxiter):
        x = (lower+upper)/2
        if callback:
            callback(iter,x,lower,upper)
        if fun(lower)*fun(x)>0:
            lower = x
        else:
            upper = x
        if np.abs(upper-lower)<tol:
            return (lower+upper)/2
    else:
        raise(RuntimeError('Failed to converge, increase maxiter'))

Use the following code to test out how the method iterates and updates the brackets.

In [ ]:
def callback_function(iter,x,lower,upper):
    if iter==0:
        print('%4s [%16s %16s] %6s'%('iter','lower','upper','diff'))
        print('-'*50)
    print('%4d [%16.12f %16.12f] %6.2e'%(iter,lower,upper,np.abs(lower-upper)))

fun = lambda x: -4*x**3+5*x+1
a,b = -3,-.5
root = bisection(fun,lower=a,upper=b,tol=1e-8,callback=callback_function)
print('Solution is ',root)

Make sure your implementation passes all the tests below.

In [ ]:
import unittest

class testsBisection(unittest.TestCase):
    """Tests for the bisection function"""

    def test1(self):
        '''solving first equation'''
        fun = lambda x: -4*x**3+5*x+1
        a,b = -3,-.5
        root = bisection(fun,lower=a,upper=b)
        self.assertTrue( np.abs(root+1) < 1e-5 )

    def test2(self):
        '''checking initial brackets'''
        fun = lambda x: -4*x**3+5*x+1
        a,b = -3,-1.5
        self.assertRaises(ValueError,bisection,fun,lower=a,upper=b)

    def test3(self):
        '''checking maxiter'''
        fun = lambda x: -4*x**3+5*x+1
        a,b = -3,-.5
        self.assertRaises(RuntimeError,bisection,fun,lower=a,upper=b,maxiter=5)

    def test4(self):
        '''checking tolerance'''
        fun = lambda x: -4*x**3+5*x+1
        a,b = -3,-.5
        root = bisection(fun,lower=a,upper=b,tol=1e-15)
        self.assertTrue( np.abs(root+1) < 1e-15 )

# this is the way to run tests
if __name__ == '__main__':
    # tweaking for Jupyter Notebook
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    # from command line
    # unittest.main()

## Task 2. Newton-Raphson method

Implement the following algorithm to solve equation $ g(x)=0 $ where $ g(x) $ is a differentiable function:

- Main inputs: function f(x), its derivative g(x), starting value  
- Additional inputs: maximum number of iterations, convergence tolerance, callback function (None by default)  


1. Initialize iteration counter at 0, set current x to the initial value,
  and loop through steps 3-6 for at most maxiter iterations:  
1. Compute the next guess x by Newton step $ x_{i+1} = x_i - \frac{f(x_i)}{g(x_i)} $  
1. Compute the change between current x and the new guess  
1. If callback function handle is passed, call this function with parameters as described in the starter code  
1. Check for convergence: if the distance between current x and new guess x is below the tolerance, exit the loop  
1. Replace the current x with the new computed guess  
1. Use for-else construct to check if the maximum number of iterations has been achieved, in which case raise
  the RuntimeError with an appropriate message.  
1. Return the last computed guess x as the solution  

## Derivation for Newton method using Taylor series expansion

$$
g(x) = \sum_{k=0}^{\infty} \frac{g^{(k)}(x_0)}{k!} (x-x_0)^k
$$

Take first two terms, use $ x $ that satisfies $ g(x)=0 $ as new approximation to the solution conditional
on the current approximation $ x_0=x_i $:

$$
0 = g(x) = g(x_i) + g'(x_i) (x_{i+1}-x_i) \quad \Rightarrow \quad x_{i+1} = x_i - \frac{g(x_i)}{g'(x_i)}
$$

In [ ]:
def newton(fun, grad, x0, tol=1e-12, maxiter=100, callback=None):
    '''Newton method to solve fun(x)=0
       Callback function arguments (iter,x,x1,err) where x1 is new guess, err = abs(x-x1)
    '''
    # write your code here

In [ ]:
def newton(fun, grad, x0, tol=1e-12, maxiter=100, callback=None):
    '''Newton method to solve fun(x)=0
       Callback function arguments (iter,x,x1,err)
    '''
    x=x0
    for iter in range(maxiter):
        x1 = x - fun(x)/grad(x)
        err = np.abs(x1-x)
        if callback:
            callback(iter,x,x1,err)
        if err < tol:
            return x1
        x = x1
    else:
        raise(RuntimeError('Failed to converge, increase maxiter'))

Use the following code to test out how the method iterates and updates the brackets.

In [ ]:
def callback_function_two(iter,x,x1,err):
    if iter==0:
        print('%4s %16s %16s %6s'%('iter','x','x1','err'))
        print('-'*50)
    print('%4d %16.12f %16.12f %6.2e'%(iter,x,x1,np.abs(x-x1)))

fun = lambda x: -4*x**3+5*x+1
grad = lambda x: -12*x**2+5
x0 = -0.5
root = newton(fun,grad,x0,tol=1e-8,callback=callback_function_two)
print('Solution is ',root)

Make sure your implementation passes all the tests below.

In [ ]:
import unittest

class testsNewton(unittest.TestCase):
    """Tests for the newton function"""

    def test1(self):
        '''solving first equation'''
        fun = lambda x: -4*x**3+5*x+1
        grad = lambda x: -12*x**2+5
        root = newton(fun,grad,-1.75)
        self.assertTrue( np.abs(root+1) < 1e-12 )

    def test2(self):
        '''checking maxiter'''
        fun = lambda x: -4*x**3+5*x+1
        grad = lambda x: -12*x**2+5
        self.assertRaises(RuntimeError,newton,fun,grad,-1.75,maxiter=2)

    def test3(self):
        '''checking tolerance'''
        fun = lambda x: -4*x**3+5*x+1
        grad = lambda x: -12*x**2+5
        root = newton(fun,grad,-1.75,tol=1e-15)
        self.assertTrue( np.abs(root+1) < 1e-15 )

# this is the way to run tests
if __name__ == '__main__':
    # tweaking for Jupyter Notebook
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    # from command line
    # unittest.main()

Investigate how different starting values affect the solution.

In [ ]:
fun = lambda x: -4*x**3+5*x+1
grad = lambda x: -12*x**2+5
x0 = [-.5,.5,1.5] # loop through these starting values

# write your code here

In [ ]:
fun = lambda x: -4*x**3+5*x+1
grad = lambda x: -12*x**2+5
x0 = [-.5,.5,1.5] # loop through these starting values
for x00 in x0:
    root = newton(fun,grad,x00,tol=1e-8)
    print('Starting value is',x00,'then solution is',root)

Compare the convergence rate between Newton-Raphson and bisection methods.  Make sure to use the same solver settings.

In [ ]:
# write your code here

In [ ]:
fun = lambda x: -4*x**3+5*x+1
grad = lambda x: -12*x**2+5
a,b = -3,-.5
x0 = (a+b)/2
print('Bisections')
root = bisection(fun,lower=a,upper=b,tol=1e-8,callback=callback_function)
print('Solution is ',root)
print('Newton-Raphson')
root = newton(fun,grad,x0,tol=1e-8,callback=callback_function_two)
print('Solution is ',root)

## Task 3. Hard equation to solve

Consider the following equation with parameters $ (p,q), p<0 $

$$
\log(x) + p \log(1-x) + q = 0, p<0
$$

Consider $ (p,q)=(-1,1) $, $ (p,q)=(-2,5) $, $ (p,q)=(-5,25) $.  For each parameter value:

1. Make a plot of the equation to show the solution  
1. Solve the equation using both bisections and Newton method to tol=1e-12  


Based on the above runs, investigate the performance of both algorithms in terms of convergence rate and robustness.

In [ ]:
# write your code here

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

for p,q in ((-1,1),(-2,5),(-5,25)):

    fun = lambda x: np.log(x) + p*np.log(1-x) + q

    x = np.linspace(0,1,1000)
    y = [fun(x) for x in x]
    plt.plot(x,y)
    plt.plot([0,1],[0,0])
    plt.show()

In [ ]:
for p,q in ((-1,1),(-2,5),(-5,25)):

    fun = lambda x: np.log(x) + p*np.log(1-x) + q
    grad = lambda x: 1/x - p/(1-x)
    a,b = 0,1
    x0 = (a+b)/2

    try:
        print('Bisections')
        root = bisection(fun,lower=a,upper=b,tol=1e-12,callback=callback_function)
        print('Solution is ',root)
    except:
        pass

    try:
        print('Newton-Raphson')
        root = newton(fun,grad,x0,tol=1e-12,callback=callback_function_two)
        print('Solution is ',root)
    except:
        pass

## Task 4. Newton methods with constraints (robust Newton)

Depending on starting value, Newton may initially shoot very far away.
When simple bounds can be set, they are useful to moderate the Newton step (at
the cost of some additional computation at each iteration).
Similarly to the bisection method, we assume that the signs of the function are
different at the bounds.

Implement the following algorithm to solve equation $ g(x)=0 $ where $ g(x) $ is a differentiable function:

- Main inputs: function f(x), its derivative g(x), lower and upper bound  
- Additional inputs: maximum number of iterations, convergence tolerance, callback function (None by default)  


1. Check that the signs of the function evaluated at the lower and upper bounds are opposite,
  otherwise raise ValueError with appropriate message.  
1. Initialize iteration counter at 0, set current interval bounds to the initial lower and upper bounds,
  set current x to the midpoint between the lower and upper bounds,
  and loop through steps 3-7 for at most maxiter iterations:  
1. Compute the next guess x by Newton step $ x_{i+1} = x_i - \frac{f(x_i)}{g(x_i)} $  
1. 
  <dl style='margin: 20px 0;'>
  <dt>If Newton step falls outside of the bounds, perform a bisection step:</dt>
  <dd>
  - Compute the sign of the function evaluated at the midpoint  
  - Move the bound of the current interval with the same sign to the midpoint and update the current interval  
  - Compute the stopping criterion as the length of the updated interval  
  - Set the next value x1 as the midpoint of the updated bound interval  
  - If callback function handle is passed, call this function with parameters as described in the starter code  
  </dd>
  
  </dl>
  
1. 
  <dl style='margin: 20px 0;'>
  <dt>If Newton step falls inside of the bounds, complete the Newton step:</dt>
  <dd>
  - Set the next value x1 equal to the next Newton guess  
  - Compute the stopping criterion as the absolute difference between x1 and current x  
  - If callback function handle is passed, call this function with parameters as described in the starter code  
  </dd>
  
  </dl>
  
1. Check for convergence: if the criterion is below the tolerance, exit the loop  
1. Replace the current x with the new value x1  
1. Use for-else construct to check if the maximum number of iterations has been achieved, in which case raise
  the RuntimeError with an appropriate message.  
1. Return the last computed x1 as the solution  

In [ ]:
def bnewton(fun, grad, lower, upper=1, tol=1e-6, maxiter=100, callback=None):
    '''Polyalgorithm that combines bisections and Newton-Raphson
       to solve fun(x)=0 within given lower and upper bounds.
       Callback function arguments (iter,itertype,x,x1,stopping_criterion) where itertype is "bisection" or "newton"
    '''
    # write your code here

In [ ]:
def bnewton(fun, grad, lower, upper, tol=1e-6, maxiter=100, callback=None):
    '''Polyalgorithm that combines bisections and Newton-Raphson
       to solve fun(x)=0 within given lower and upper bounds.
       Callback function arguments (iter,itertype,x,x1,stopping_criterion) where itertype is "bisection" or "newton"
    '''
    sign_lower = np.sign(fun(lower))
    sign_upper = np.sign(fun(upper))
    if sign_lower*sign_upper>0:
        raise(ValueError('Bad initial lower and upper limits'))
    x = (lower+upper)/2
    for iter in range(maxiter):
        newt = x - fun(x)/grad(x) # Newton step
        if newt<lower or newt>upper:
            # bisection step
            if np.sign(fun(x))*sign_lower > 0:
                lower = x  # and the lower sign remains
            else:
                upper = x  # and the upper sign remains
            x1 = (lower+upper)/2
            stopping = np.abs(upper-lower)
            if callback:
                callback(iter,'bisect',x,x1,stopping)
        else:
            x1 = newt
            stopping = np.abs(x1-x)
            if callback:
                callback(iter,'newton',x,x1,stopping)
        x = x1
        if stopping<tol:
            return x1
    else:
        raise(RuntimeError('Failed to converge, increase maxiter'))

Make sure your implementation passes all the tests below.

In [ ]:
import unittest

class testsBnewton(unittest.TestCase):
    """Tests for the newton function"""

    def test1(self):
        '''solving first equation'''
        fun = lambda x: -4*x**3+5*x+1
        grad = lambda x: -12*x**2+5
        a,b = -3,-.5
        root = bnewton(fun,grad,a,b)
        self.assertTrue( np.abs(root+1) < 1e-12 )


    def test2(self):
        '''checking initial brackets'''
        fun = lambda x: -4*x**3+5*x+1
        grad = lambda x: -12*x**2+5
        a,b = -3,-1.5
        self.assertRaises(ValueError,bnewton,fun,grad,a,b)

    def test3(self):
        '''checking maxiter'''
        fun = lambda x: -4*x**3+5*x+1
        grad = lambda x: -12*x**2+5
        a,b = -3,-.5
        self.assertRaises(RuntimeError,bnewton,fun,grad,a,b,maxiter=5)

    def test4(self):
        '''checking tolerance'''
        fun = lambda x: -4*x**3+5*x+1
        grad = lambda x: -12*x**2+5
        a,b = -3,-.5
        root = bnewton(fun,grad,a,b,tol=1e-15)
        self.assertTrue( np.abs(root+1) < 1e-15 )


# this is the way to run tests
if __name__ == '__main__':
    # tweaking for Jupyter Notebook
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    # from command line
    # unittest.main()

Finally, solve the hard equation from Task 3 using the robust Newton, and compare the convergence rate to the bisection algorithm.

In [ ]:
# write your code here

In [ ]:
def callback_function_three(iter,type,x,x1,err):
    if iter==0:
        print('%4s %8s %16s %16s %6s'%('iter','type','x','x1','err'))
        print('-'*57)
    print('%4d %8s %16.12f %16.12f %6.2e'%(iter,type,x,x1,np.abs(x-x1)))


for p,q in ((-1,1),(-2,5),(-5,25)):

    fun = lambda x: np.log(x) + p*np.log(1-x) + q
    grad = lambda x: 1/x - p/(1-x)
    a,b = 0,1

    print('Bisections')
    root = bisection(fun,a,b,tol=1e-12,callback=callback_function)
    print('Solution is ',root)

    print('Robust Newton-Raphson')
    root = bnewton(fun,grad,a,b,tol=1e-12,callback=callback_function_three)
    print('Solution is ',root)